quelle : https://www.govdata.de/suche/daten/donau-ries-lkr-apfel-und-birnen

In [33]:
import requests

url = "https://fms.donau-ries.de/formcycle/datenabfragedb?mandantName=LRA-Donau-Ries&name=aepfel-und-birnen"

# Anfrage schicken
response = requests.get(url)

# prüfen ob alles ok war
if response.status_code == 200:
    data = response.json()   # JSON direkt als Python-Dict/Liste
    #print(data['result'])
else:
    print("Fehler beim Abrufen:", response.status_code)



In [34]:
from bs4 import BeautifulSoup
ls_keys = ["geschichte", "frucht", "reifezeit", "standort", "anfälligkeit", "blüte", "sonstiges" ]
alle_sorten = {}
number  = 0
for eintrag in data["result"]:
    alias = eintrag["alias"]
    
    html_text = eintrag["text_inhalt"]
    #quellen = eintrag.get("quellen", "")

    soup = BeautifulSoup(html_text, "html.parser")

    sorten_dict = {}
    current_key = None
    #print(soup)
    for tag in soup.find_all("p"):
        text = tag.get_text(strip=True)
        #print(tag)
        # Neue Sektion wenn <strong> drin ist
        if tag.find("strong"):
            current_key = text.replace(":", "").lower()
            if current_key=="früchte":
                current_key="frucht"
            if current_key=="früchte":
                current_key="frucht"
            sorten_dict[current_key] = ""
        else:
            if current_key:
                if sorten_dict[current_key]:
                    sorten_dict[current_key] += " " + text
                else:
                    sorten_dict[current_key] = text
    sorten_dict["ind"] = number
    alle_sorten[alias] = sorten_dict
    number += 1
#print(alle_sorten["blauapfel"])
#alle_sorten["horneburger-pfannkuchenapfel"]

#alle_sorten["suesskirsche-skeena"]

In [35]:

LS_KEYS = ["geschichte", "frucht", "reifezeit", "standort", "anfälligkeit", "blüte", "sonstiges", "ind"]
alle_sorten_cleaned  = {}
sorten_cleaned  = {}
for key, value in alle_sorten.items():
    
    for ke, val in value.items():
        if ke in LS_KEYS:
            sorten_cleaned[ke]= val
            continue
        else:
            for wort in LS_KEYS:
                if ke.startswith(wort):
                    sorten_cleaned[wort] =  ke[len(wort)::] 
                else:
                    continue
    alle_sorten_cleaned[key] = sorten_cleaned
    sorten_cleaned  = {}



In [36]:
import pandas as pd
df = pd.DataFrame(alle_sorten_cleaned)
df  = df.T
lss = df[df["anfälligkeit"].notna()]["anfälligkeit"].tolist()
lsn = df[df["anfälligkeit"].notna()]["ind"].tolist()

anf = dict(zip(lsn,lss))

In [37]:
import re
Feuerbrand_filtered = {}
for i, s in anf.items():
    if "feuer" in anf[i].lower():
        for st in re.split(r"[,.]", anf[i]):
           if "feuer" in st.lower():
                Feuerbrand_filtered[i] = st 

In [38]:
Feuerbrand_filtered
# gibt es einen unterschied zwischen gefährdet und anfällig?
# Wie kann man das Feuerbrandgefahr einstufen?

{0: 'Feuerbrand',
 1: 'Stark Feuerbrand gefährdet',
 2: 'Stark anfällig für Feuerbrand',
 3: ' Mittelanfällig für Feuerbrand',
 4: 'Mittelstark anfällig für Feuerbrand',
 8: 'Stark Feuerbrand gefährdet',
 9: 'Geringe Feuerbrandanfälligkeit',
 13: ' Wenig feuerbrandgefährdet',
 19: ' Mittelstark anfällig für Feuerbrand',
 22: 'Stark Feuerbrandgefährdet',
 23: 'Stark Feuerbrand',
 24: 'Stark Feuerbrand gefährdet',
 25: ' Mittelstark Feuerbrand gefährdet',
 29: 'Stark feuerbrand- und mehltauanfällig',
 30: ' Widerstandsfähig gegen Feuerbrand',
 31: 'Stark Feuerbrand gefährdet',
 32: 'Stark Feuerbrand und Obstbaumkrebs gefährdet',
 35: 'Gering Feuerbrand gefährdet',
 36: 'Mittelstark Feuerbrand gefährdet',
 37: 'Mittelstark Feuerbrand gefährdet',
 41: 'Geringe Feuerbrandgefahr',
 43: 'Stark Feuerbrandgefährdet',
 44: 'Bei Temperaturschwankungen auf Feuerbrand achten',
 45: 'Feuerbrand gefährdet',
 50: 'Wenig Feuerbrand und Schorf gefährdet',
 53: 'Mittelstark Feuerbrand gefährdet',
 55: ' 

In [39]:
df.head(5)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 173 entries, alkmene to birne-uta
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   geschichte    166 non-null    object
 1   frucht        172 non-null    object
 2   reifezeit     162 non-null    object
 3   standort      162 non-null    object
 4   anfälligkeit  161 non-null    object
 5   blüte         140 non-null    object
 6   sonstiges     138 non-null    object
 7   ind           173 non-null    object
dtypes: object(8)
memory usage: 16.2+ KB
